# Constrained shape design of a transonic inviscid wing at a cte. $C_{l}$ [1]

## 1. Introduction 

The following tutorial will walk the reader through the steps required when performing 3D shape design using SU2. The optimisation problem has been first executed as the regime was inviscid and subsequently introducing a finite Reynolds number and, therefore, a turbulence model. The mesh being employed is the one SU2 tutorials collection provides with, which is extremely coarse and reduces whether accuracy and computational effort. As design variables, free-form deformation (FFD) approach shall be used. In this approach, a lattice of control points making up a bounding box are placed around the geometry, and the movement of these control points smoothly deforms the surface shape of the geometry contained inside [1]. 

The design's targets is to scope out the design space in order to come up with the geometrical shape minimizing the drag by constraining certain sections thickness (further details shall be given afterwards).

The mesh consists of a far-field boundary divided in three surfaces (XNORMAL_FACES, ZNORMAL_FACES, YNORMAL_FACES), an Euler wall (flow tangency) divided into three surfaces (UPPER_SIDE, LOWER_SIDE, TIP), and a symmetry plane (SYMMETRY_FACE) [1]. 

The optimisation running time has resulted in more than **3 days** on 1 core for a total of **20** iterations and **46** gradients evaluation.

###############################################################################################################################

## 2. Configuration file 

As for the configuration file regards, the only actual difference revealed is the FFD, i.e. the design variables, as well as the running commands.


In [3]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_Viscid_FFD.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                                                              %
% SU2 configuration file                                                       %
% Case description: Optimization case for the ONERA M6 wing in viscid flow     %
% Author: Schioppa Aldo	         	       	                               %
% Institution: ISAE-Supaero	                                               %
% Date: August, 2022                                                           %
% File Version: Newest                                                         %
%                                                                              %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%
% -------------------- FREE-FORM DEFORMATION PARAMETERS -----------------------%
%
% Tolerance of the Free-Form Deformation point inversion
FFD_TOLERANCE= 1E-10
%
% Maximum number of iterations in t

The current implementation requires each FFD box to be a quadrilaterally-faced hexahedron (6 faces, 12 edges, 8 vertices), thus requiring the 8 corner points of the box and the polynomial degree the user would like to impose along each coordinate direction *(x,y,z)* in order to create the complete lattice of control points. In fact, it is known that the *N_CONTROL_POINTS = POLYNOMIAL_DEGREE + 1*. The FFD edges' degrees are specified in *FFD_DEGREE*. In this case, the total number of points is 198 (N_CONTROL_POINTS_X * N_CONTROL_POINT_Y * N_CONTROL_POINTS_Z, 11 * 9 * 2, respectively). Generally speaking, setting FFDs is not complex as long as simple rules and steps are followed. First and foremost, the user should observe that only 3 parameters (*FFD_DEFINITION*, *FFD_DEGREE*, *DV_MARKER*) are necessary to set, as *DV_PARAM*, *DV_KIND*, *FFD_CONTINUITY*, *FFD_TOLERANCE*, *FFD_ITERATIONS* being fixed. Let's walk through each and every of them:

* **FFD_DEFINITION**: In this section, the 8 (4, for 2D geometries) vertices are obligatorily to have to be defined. The first string represents the FFD box tag name, in this case *WING*. The order of how the points are written is crucial; The FFD-box points are addressed via i-j-k indices assuming FFD-edges to be aligned with cartesian axes. The rule is very simple and can be generalised in the following way: 

**"Beginning from the lowest possible *z*-plane, choose the lowest *x-y-z* point as starting point and proceed counter-clockwise."**

* **FFD_DEGREE**: Determines the number of FFD-Box points per i-j-k-index. The degree plus 1 gives the number of points employed.

* **DV_MARKER**: Boundary markers that are going to be deformed by the FFD-Box. Note that the Mesh deformation in SU2 is a 2-stage process:

1. The FFD-Box deforms only the boundary mesh nodes that are inside of the initial FFD-Box. These nodes are stored at the bottom of the .su2 mesh that contain the FFD-Box.
2. The deformed boundary from the previous step is now boundary condition for a Linear Elasticity style volume mesh morpher [2].

Furthermore, the FFD capabilities within SU2 also feature a nifty technique to automatically ensure that the mesh does not end up with any jumps or kinks in the deformed geometry. The latter can be controlled by requesting continuity in the 1st or 2nd derivative of the surface with the *FFD_CONTINUITY* option. In short, the code will automatically detect when a face of the FFD box intersects the geometry, and it shall hold the control points fixed on that face (*1ST_DERIVATIVE*) or the points on the face as well as one slice of adjacent control points (*2ND_DERIVATIVE*). **Note that these control points shall be held fixed during design cycles even if you specify them in your design variable list** [2].

To conclude, once the FFD-box has been defined, it is necessary to generate a new mesh containing the box information. To do so, the following line must be run in the command prompt **BEFORE** running the adjoint.

In [6]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_Box_Info.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

SU2_DEF <config-file>.cfg


After completing the FFD mapping process, a mesh file named “mesh_out.su2” (by default) is now in the directory. Rename the file and pass it onto the configuration file. Note that this new mesh file contains all the details of the FFD method. **If the user was to face particular case stalls or be thrown errors during the creation of the box, *FFD_TOLERANCE* and *FFD_ITERATIONS* can be adjusted to achieve convergence of the algorithm.**

Lastly, the final step consist in truly generating the design variables undergoing to the desing procedure, i.e. the 198 points mentioned beforehand. It is possible to automatically have them popped out onto the command prompt through the following line, involving the **set_ffd_design_var.py** script that is shipped with the other Python utilities with the source code:

In [8]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_Design_Variables.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

$ python set_ffd_design_var.py -i <DEGREE_X> -j <DEGREE_Y> -k <DEGREE_Z> -b <FFD-Box_TAG> -m '<MARKER_1, MARKER_2, ..., MARKER_N>'

In this case:

$ python set_ffd_design_var.py -i 10 -j 8 -k 1 -b WING -m 'UPPER_SIDE, LOWER_SIDE, TIP'




**WARNING**: Avoid the following error if arose. The only purpose of launching the script is to retrieve the design variables.

In [9]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_ERROR.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

% FFD_NACELLE (RHO)
Traceback (most recent call last):
  File "D:\Downloads\Viscid_3D_Constrained_ONERAM6\set_ffd_design_var.py", line 123, in <module>
    for jIndex in range(1+options.jOrder/2):
TypeError: 'float' object cannot be interpreted as an integer


The output must be pasted onto the configuration file, line *DEFINITION_DV*.

**WARNING**: The *DEFINITION_DV* should look like:

In [11]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_DV.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

DEFINITION_DV= ( 11, 1.0 | 'UPPER_SIDE, | WING, 0, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 1, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 2, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 3, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 4, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 5, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 6, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 7, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 8, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 9, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 10, 0, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 0, 1, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 1, 1, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 2, 1, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 3, 1, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 4, 1, 0, 1.0, 0.0, 0.0 ); ( 11, 1.0 | 'UPPER_SIDE, | WING, 5, 1, 0

in other words, link them adiacently. It is here recalled that, at the very end of the list, the semi-colon is not present.

Generally, it is also important to adjust the scale of the *OPT_CONSTRAINT*; this scale factor is an effective method to control how much the violated constraint is going to change the objective function gradient (**push factor**). Along with **OPT_GRADIENT_FACTOR** and **OPT_RELAX_FACTOR** (see [2][3]), the adjustment of this parameter is important to fully exploit the design space possibilities of the Python gradient based optimizers.



###############################################################################################################################

In [14]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_MULTI_GRIDS.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                                                              %
% SU2 configuration file                                                       %
% Case description: Optimization case for the ONERA M6 wing in viscid flow     %
% Author: Schioppa Aldo	         	       	                               %
% Institution: ISAE-Supaero	                                               %
% Date: August, 2022                                                           %
% File Version: Newest                                                         %
%                                                                              %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%
% -------------------------- MULTIGRID PARAMETERS -----------------------------%
%
% Multi-Grid Levels (0 = no multi-grid)
MGLEVEL= 0
%
% Multi-grid cycle (V_CYCLE, W_CYCLE, FULLMG_CYCLE)
MGCYCLE= w

A small digression concerning **MULTI-GRID** approach is hereby described. In numerical analysis, a multigrid method (MG method) is an algorithm for solving elliptic partial differential equations in two or more dimensions using a hierarchy of discretizations. They are an example of a class of techniques called multiresolution methods, very useful in problems exhibiting multiple scales of behavior. For example, many basic relaxation methods (non/linear systems resolution method) exhibit different rates of convergence for short- and long-wavelength components [5]. To profusely understand, it is decisive to introduce the concept of short/long-wavelength error. 

Let's consider a cell and its immediate adjacent neighbors. Through FVM method (for example) on this cell, the equations are solved but small error (the residual) is present. **This is the short wavelength error**.
Now let's consider a cell in the inside the domain (say the center). The method employed ends up with a solution for this interior cell, whatever that solution is. Although it may (or may not) satisfy the local flux balances, it may be inconsistent with the boundary conditions imposed on the domain surfaces. **This is the long wavelength error**.

For a coarse mesh, **the longest possible wavelengths of error components (i.e. those of the order of the domain size) are just within the short-wavelength range of the mesh and, hence, all error components reduce rapidly (hence, the numerical *RESIDUALS*, therefore converging faster). On the finer meshes, however, the longest error wavelengths are progressively further outside the short-wavelength range, so the decay is slow**. Summarizing, in general the grid acts like a filter and a smoother. A coarse mesh is able to apply a "filtering" effect on the components of the errors. When the mesh is sufficiently coarse, also the components at low wavelenghts are smoothed. Conversely, for a fine mesh (long-wavelenght) you have a very slow decay of the error. The practical outcome of this phenomenon is that, without multi-grid algorithms or convergence accelerators, you need more and more iterations to converge, not only because of the refinement already present, but also because of slowness with which the error decays underneath an acceptable treshold. 

The main idea of multigrid is to accelerate the convergence of a basic iterative method (known as relaxation, which generally reduces short-wavelength error) by a global correction of the fine grid solution approximation from time to time, accomplished by solving a coarse problem. The coarse problem, while cheaper to solve, is similar to the fine grid problem in which it also has short- and long-wavelength errors. This recursive process is repeated until a grid is reached where the cost of direct solution there is negligible compared to the cost of one relaxation sweep on the fine grid, reaching faster convergence [5].


###############################################################################################################################

In [4]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_GEO_EVALUATION_PAR.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                                                              %
% SU2 configuration file                                                       %
% Case description: Optimization case for the ONERA M6 wing in viscid flow     %
% Author: Schioppa Aldo	         	       	                               %
% Institution: ISAE-Supaero	                                               %
% Date: August, 2022                                                           %
% File Version: Latest                                                         %
%                                                                              %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%
% ----------------------- GEOMETRY EVALUATION PARAMETERS ----------------------%
%
% Marker(s) of the surface where geometrical based function will be evaluated
GEO_MARKER= ( UPPER_SIDE, LOWER_SIDE,

In the *GEOMETRY EVALUATION PARAMETERS* section (3D), the option *WING* in the *GEO_DESCRIPTION* is necessary for the optimisation problem to be well defined. *GEO_LOCATION_STATIONS* sets up the wing's section to be analysed and it is decisive for the geometrical constraints being appliable upon the wing. It is advisable to not edit the rest.

###############################################################################################################################

In [6]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_CONSTRAINTS.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                                                              %
% SU2 configuration file                                                       %
% Case description: Optimization case for the ONERA M6 wing in viscid flow     %
% Author: Schioppa Aldo	         	       	                               %
% Institution: ISAE-Supaero	                                               %
% Date: August, 2022                                                           %
% File Version: Latest                                                         %
%                                                                              %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% --------------------- OPTIMAL SHAPE DESIGN DEFINITION -----------------------%
%
% Available flow based objective functions or constraint functions
%    DRAG, LIFT, SIDEFORCE, EFFICIENCY, BUFFET, 
%

In the final section of the configuration file the constraints are set. In this case, observe that, according to what has been defined in *GEO_LOCATIONS_STATIONS*, to fix airfoils' thickness (for instance) at certain stations, the following line is necessary:

**STATION#_THICKNESS**

where # is the station index defined in *GEO_LOCATION_STATIONS* indeed. The rest has been already discussed in [3][4].

###############################################################################################################################

The current case configuration file, albeit inviscid, is easily generalisable to viscid cases by adding in the following section:

In [2]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/ONERAM6_VISCID_SECTION.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                                                                              %
% SU2 configuration file                                                       %
% Case description: Optimization case for the ONERA M6 wing in viscid flow     %
% Author: Schioppa Aldo	         	       	                               %
% Institution: ISAE-Supaero	                                               %
% Date: August, 2022                                                           %
% File Version: Latest                                                         %
%                                                                              %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%
% -------------------- COMPRESSIBLE FREE-STREAM DEFINITION --------------------%
%
% Mach number (non-dimensional, based on the free-stream values)
MACH_NUMBER= 0.8395
%
% Angle of attack (degrees)


Clearly, it is obligatory to introduce prism layer into the mesh. Nonetheless, the procedure is unchanged.

# Bibliography 

In [15]:
f=open('D:/SU2/Adjoint/Viscid_3D_Constrained_ONERAM6/TXT_FILES/Bibliography.txt', 'r')
cfg_file=f.read()
print(cfg_file)
f.close()


[1] SU2's cite: https://su2code.github.io/tutorials/Inviscid_3D_Constrained_ONERAM6/

[2] SU2's cite: https://su2code.github.io/tutorials/Species_Transport/

[3] Schioppa, Aldo. "Adjoint Method in SU2", August 2022, Summer Internship, ISAE-SUPAERO, Jupyter Notebook.

[4] Schioppa, Aldo. "Steady viscid and compressible/incompressible constrained optimisation at constant Cl", August 2022, Summer Internship, ISAE-SUPAERO, Jupyter Notebook.

[5] Wikipedia: https://en.wikipedia.org/wiki/Multigrid_method
